In [ ]:
from time import time
import casadi as ca
import numpy as np
from casadi import sin, cos, tan, pi

In [ ]:
def shift_timestep(step_horizon, t0, state_init, u, f):
    f_value = f(state_init, u[:, 0])
    next_state = ca.DM.full(state_init + (step_horizon * f_value))

    t0 = t0 + step_horizon
    u0 = ca.horzcat(
        u[:, 1:],
        ca.reshape(u[:, -1], -1, 1)
    )

    return t0, next_state, u0


def DM2Arr(dm):
    return np.array(dm.full())

In [ ]:
_T = 2.0   # Prediction horizon for MPC and local planner
_dt = 0.2 # Sampling time step for MPC and local planner
_N = int(_T/_dt)
sim_time = 20
_N

In [ ]:
# bluerov parameters
x_g = 0
y_g = 0
z_g = 0
x_b = 0
y_b = 0
z_b = -0.01

#added mass coefficients
X_ud = 6.356673886738176
Y_vd = 7.120600295756984
Z_wd = 18.686326861534997
K_pd = 0.185765630747592
M_qd = 0.134823349429660
N_rd = 0.221510466644690
#drag 
Xu = {"Linear": 13.7, "NonLinear": 141}
Yv = {"Linear": 0, "NonLinear": 217}
Zw = {"Linear": 33, "NonLinear": 190}
Kp = {"Linear": 0, "NonLinear": 1.192}
Mq = {"Linear": 0.8, "NonLinear": 0.470}
Nr = {"Linear": 0, "NonLinear": 1.5}


In [ ]:
m = 13.5 # Mass of the Robot

I_x, I_y, I_z = 0.26, 0.23, 0.37 # Mass Moments of Inertia

L_h, L_w, L_l = 0.378, 0.575, 0.457 

A_F, A_S, A_T = 0.0877, 0.1131, 0.2049

Volume = 0.0135

Ic = np.array([[0.26, 0, 0],
      [0, 0.23, 0],
      [0, 0, 0.37]])

Ap_F, Ap_S, Ap_T = 0.1727, 0.2174, 0.2628
W = m * 9.81
B = 1000 * Volume * 9.81

In [ ]:
_s_dim = 12
_u_dim = 6
# matrix containing all states over all time steps +1 (each column is a state vector)
_X = ca.SX.sym('X', _s_dim, _N + 1)

# matrix containing all control actions over all time steps (each column is an action vector)
_U = ca.SX.sym('U', _u_dim, _N)

# coloumn vector for storing initial state and target state
_P = ca.SX.sym('P', _s_dim + _s_dim)

# state weights matrix (Q_X, Q_Y, Q_THETA)
Q = ca.diagcat(
    100, 100, 100,
    10, 10, 10, 
    10, 10, 10,
    10, 10, 10
)

# controls weights matrix
R = ca.diagcat(0.1,0.1,0.1,0.1,0.1,0.1)
# initial state and control action
_bluerov_s0 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
_bluerov_u0 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Reference state and control action
_bluerov_sref = [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
_thrust_min = -20
_thrust_max = 20


In [ ]:
# # # # # # # # # # # # # # # # # # # 
# ---------- Input States -----------
# # # # # # # # # # # # # # # # # # # 

px, py, pz = ca.SX.sym('px'), ca.SX.sym('py'), ca.SX.sym('pz')
#
phi, theta, psi = ca.SX.sym('phi'), ca.SX.sym('theta'), ca.SX.sym('psi')
#
u, v, w = ca.SX.sym('u'), ca.SX.sym('v'), ca.SX.sym('w')
p, q, r = ca.SX.sym('p'), ca.SX.sym('q'), ca.SX.sym('r')

# -- conctenated vector
_x = ca.vertcat(px, py, pz, phi, theta, psi, u, v, w, p, q, r) 

# # # # # # # # # # # # # # # # # # # 
# --------- Control Command ------------
# # # # # # # # # # # # # # # # # # #

X, Y, Z, K, M, N = ca.SX.sym('X'), ca.SX.sym('Y'), ca.SX.sym('Z'),\
    ca.SX.sym('K'), ca.SX.sym('M'), ca.SX.sym('N')

# -- conctenated vector
_u = ca.vertcat(X, Y, Z, K, M, N)

In [ ]:
#Rigid-body mass
M_RB = ca.DM([
    [m, 0, 0, 0, 0, 0],
    [0, m, 0, 0, 0, 0],
    [0, 0, m, 0, 0, 0],
    [0, 0, 0, I_x, 0, 0 ],
    [0, 0, 0, 0, I_y, 0 ],
    [0, 0, 0, 0, 0, I_z ]
    ])
#added mass
M_A = -1 * ca.DM([[X_ud, 0, 0, 0, 0, 0],
              [0, Y_vd, 0, 0, 0, 0],
              [0, 0, Z_wd, 0, 0, 0],
              [0, 0, 0, K_pd, 0, 0 ],
              [0, 0, 0, 0, M_qd, 0 ],
              [0, 0, 0, 0, 0, N_rd ]])
M = M_RB + M_A
M

In [ ]:
# # # # # # # # # # Coriolis # # # # # # # # #
C_RB = ca.vertcat(
    ca.horzcat(0, 0, 0, 0, m*w, -m*v ),
    ca.horzcat(0, 0, 0, -m*w, 0, m*u ),
    ca.horzcat(0, 0, 0, m*v, -m*u, 0 ),
    ca.horzcat(0, m*w, -m*v, 0, -I_z*r, -I_y*q ),
    ca.horzcat(-m*w, 0, m*u, I_z*r, 0, I_x*p ),
    ca.horzcat(m*v, -m*u, 0, I_y*q, -I_x*p, 0 )
    )

# added mass Coriolis
C_A = ca.vertcat(
    ca.horzcat(0, 0, 0, 0, -Z_wd*w, Y_vd*v),
    ca.horzcat(0, 0, 0, Z_wd*w, 0,-X_ud*u), 
    ca.horzcat(0, 0, 0, -Y_vd*v, X_ud*u, 0),
    ca.horzcat(0, -Z_wd*w, Y_vd*v, 0, -N_rd*r, M_qd*q),
    ca.horzcat(Z_wd*w, 0, -X_ud*u, N_rd*r, 0, -K_pd*p),
    ca.horzcat(-Y_vd*v, X_ud*u, 0, -M_qd*q, K_pd*p, 0)
    )

C = C_RB + C_A
C

In [ ]:
# # # # # # # # # # Drag # # # # # # # # #
D_lin = ca.DM([
    [Xu["Linear"], 0, 0, 0, 0, 0],
    [0, Yv["Linear"], 0, 0, 0, 0],
    [0, 0, Zw["Linear"], 0, 0, 0],
    [0, 0, 0, Kp["Linear"], 0, 0],
    [0, 0, 0, 0, Mq["Linear"], 0],
    [0, 0, 0, 0, 0, Nr["Linear"]]
])
    
D_nonlin = ca.vertcat(
    ca.horzcat(Xu["NonLinear"] * u, 0, 0, 0, 0, 0 ),
    ca.horzcat(0, Yv["NonLinear"] * v, 0, 0, 0, 0 ),
    ca.horzcat(0, 0, Zw["NonLinear"] * w, 0, 0, 0 ),
    ca.horzcat(0, 0, 0, Kp["NonLinear"] * p, 0, 0 ),
    ca.horzcat(0, 0, 0, 0, Mq["NonLinear"] * q, 0 ),
    ca.horzcat(0, 0, 0, 0, 0, Nr["NonLinear"] * r )       
)

D = D_lin + D_nonlin
D

In [ ]:
G = ca.vertcat(
    (W - B) * sin(theta),
    -(W - B) * cos(theta) * sin(phi),
    -(W - B) * cos(theta) * cos(phi),
    -(y_g*W - y_b*B)*cos(theta)*cos(phi) + (z_g*W - z_b*B)*cos(theta)*sin(phi),
    (z_g*W - z_b*B)*sin(theta) + (x_g*W - x_b*B)*cos(theta)*cos(phi),
    -(x_g*W - x_b*B)*cos(theta)*sin(phi) - (y_g*W - y_b*B)*sin(theta)
    )
G

In [ ]:
J2 = ca.vertcat(
    ca.horzcat(1, sin(phi)*tan(theta), cos(phi)*tan(theta)),
    ca.horzcat(0, cos(phi), -sin(phi)),
    ca.horzcat(0, sin(phi)/cos(theta), cos(phi)/cos(theta))
)

J1 = ca.vertcat(
    ca.horzcat(cos(psi)*cos(theta), -sin(psi)*cos(phi) + cos(psi)*sin(theta)*sin(phi), sin(psi)*sin(phi) + cos(psi)*cos(phi)*sin(theta)),
    ca.horzcat(sin(psi)*cos(theta), cos(psi)*cos(phi) + sin(phi)*sin(theta)*sin(psi), -cos(psi)*sin(phi) + sin(theta)*sin(psi)*cos(phi)),
    ca.horzcat(-sin(theta), cos(theta)*sin(phi), cos(theta)*cos(phi))
)

J = ca.vertcat(
    ca.horzcat(J1, ca.SX.zeros(3,3)),
    ca.horzcat(ca.SX.zeros(3,3), J2))
J

In [ ]:
eta_dot = J @ _x[6:]
v_dot =  ca.inv( M ) @ (_u  - C @ _x[6:] - D @ _x[6:] - G )

In [ ]:
x_dot = ca.vertcat(
            eta_dot,
            v_dot
        )
f = ca.Function('f', [_x, _u], [x_dot], ['x', 'u'], ['ode'])

In [ ]:
cost_fn = 0  # cost function
g = _X[:, 0] - _P[:_s_dim]  # constraints in the equation
lbx = []
ubx = []
x_bound = ca.inf
x_min = [-x_bound for _ in range(_s_dim)]
x_max = [+x_bound for _ in range(_s_dim)]

lbx += x_min
ubx += x_max
u_min = [_thrust_min for _ in range(_u_dim)]
u_max = [_thrust_max for _ in range(_u_dim)]

# runge kutta
for k in range(_N):
    st = _X[:, k]
    con = _U[:, k]
    cost_fn = cost_fn \
        + (st - _P[_s_dim:]).T @ Q @ (st - _P[_s_dim:]) \
        + con.T @ R @ con
    st_next = _X[:, k+1]
    k1 = f(st, con)
    k2 = f(st + _dt/2*k1, con)
    k3 = f(st + _dt/2*k2, con)
    k4 = f(st + _dt * k3, con)
    st_next_RK4 = st + (_dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
    g = ca.vertcat(g, st_next - st_next_RK4)
    
    lbx += u_min
    ubx += u_max
    lbx += x_min
    ubx += x_max
    


OPT_variables = ca.vertcat(
    _X.reshape((-1, 1)),   # Example: 3x11 ---> 33x1 where 3=states, 11=N+1
    _U.reshape((-1, 1))
)
nlp_prob = {
    'f': cost_fn,
    'x': OPT_variables,
    'g': g,
    'p': _P
}

opts = {
    'ipopt': {
        'max_iter': 2000,
        'print_level': 0,
        'acceptable_tol': 1e-8,
        'acceptable_obj_change_tol': 1e-6
    },
    'print_time': 0
}

solver = ca.nlpsol('solver', 'ipopt', nlp_prob, opts)

In [ ]:
args = {
    'lbg': ca.DM.zeros((_s_dim*(_N+1), 1)),  # constraints lower bound
    'ubg': ca.DM.zeros((_s_dim*(_N+1), 1)),  # constraints upper bound
    'lbx': lbx,
    'ubx': ubx
}

t0 = 0
state_init = ca.DM(_bluerov_s0)        # initial state
state_target = ca.DM(_bluerov_sref)  # target state

# xx = DM(state_init)
t = ca.DM(t0)

u0 = ca.DM.zeros((_u_dim, _N))  # initial control
X0 = ca.repmat(state_init, 1, _N+1)         # initial state full


mpc_iter = 0
cat_states = DM2Arr(X0)
cat_controls = DM2Arr(u0[:, 0])
times = np.array([[0]])

In [ ]:
main_loop = time()  # return time in sec
while (ca.norm_2(state_init - state_target) > 1e-1) and (mpc_iter * _dt < sim_time):
    t1 = time()
    args['p'] = ca.vertcat(
        state_init,    # current state
        state_target   # target state
    )
    # optimization variable current state
    args['x0'] = ca.vertcat(
        ca.reshape(X0, _s_dim*(_N+1), 1),
        ca.reshape(u0, _u_dim*_N, 1)
    )

    sol = solver(
        x0=args['x0'],
        lbx=args['lbx'],
        ubx=args['ubx'],
        lbg=args['lbg'],
        ubg=args['ubg'],
        p=args['p']
    )

    u = ca.reshape(sol['x'][_s_dim * (_N + 1):], _u_dim, _N)
    X0 = ca.reshape(sol['x'][: _s_dim * (_N+1)], _s_dim, _N+1)

    cat_states = np.dstack((
        cat_states,
        DM2Arr(X0)
    ))

    cat_controls = np.vstack((
        cat_controls,
        DM2Arr(u[:, 0])
    ))
    t = np.vstack((
        t,
        t0
    ))

    t0, state_init, u0 = shift_timestep(_dt, t0, state_init, u, f)

    # print(X0)
    X0 = ca.horzcat(
        X0[:, 1:],
        ca.reshape(X0[:, -1], -1, 1)
    )

    # xx ...
    t2 = time()
    print(mpc_iter)
    print(t2-t1)
    times = np.vstack((
        times,
        t2-t1
    ))

    mpc_iter = mpc_iter + 1

main_loop_time = time()
ss_error = ca.norm_2(state_init - state_target)

print('\n\n')
print('Total time: ', main_loop_time - main_loop)
print('avg iteration time: ', np.array(times).mean() * 1000, 'ms')
print('final error: ', ss_error)